In [22]:
from bs4 import BeautifulSoup as bs
import requests
from urllib.request import urlopen
import re #regex
import pandas as pd
import nltk
from nltk.corpus import words
nltk.download('words')

[nltk_data] Downloading package words to /Users/srijon1/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [23]:
englishCorpus = set(words)


TypeError: 'LazyCorpusLoader' object is not iterable

In [4]:
def getLyrics(url):
    lyrics = []
    soup = bs(urlopen(url), 'html.parser')
    songDiv=  soup.find(class_= 'feat').next_sibling.next_sibling.next_sibling.next_sibling.next_sibling
    print(songDiv)
    for child in songDiv.descendants:
        if child.text !='\n' and child.text != '':
            newline = r'[\n]'
            cleaned = re.sub( newline, '', child.text)
            titleTag = r'[\r]'
            cleaned = re.sub( titleTag, '', cleaned)
            lyrics.append(cleaned)
    return lyrics
    

In [11]:
url = "https://www.azlyrics.com/b/bollywood.html"
ROOT_URL = "https://www.azlyrics.com"
response = requests.get(url)
html = response.content # the raw html will be unparsable to humans 

soup = bs(html, "lxml") # uses lxml library to parse the raw html 
allAlbumDivs = soup.find_all(class_="listalbum-item")

allSongs = []
for listalbum in allAlbumDivs[:10]: 
    song_url = listalbum.find('a').get('href')
    url = ROOT_URL + song_url
    lyrics = getLyrics(url)
    allSongs.append(lyrics)
print(allSongs)


[]


In [73]:
# scrape from local dataset 
def cleanText(song):
#     (?<=a).*(?=b)
    pattern = r'(?<=-).*(?=-)'
    cleaned = re.sub( pattern, '', song)
    cleaned = re.sub(r'-', '', cleaned)
    cleaned = re.sub(r'\.','', cleaned)
    cleanedList = cleaned.split('\n')
    cleanedList = [x for x in cleanedList if x!='']
    return cleanedList

In [74]:
def createTextFile(corpusName, data):
    f = open(corpusName,"w")
    print('cn', corpusName)
    for song in data:
        cleanedSong = cleanText(song)
        for line in cleanedSong: 
            f.write(line+'\n')
    f.close()

In [36]:
data = pd.read_csv('hindi-lyrics.csv')
df = pd.DataFrame(data)

allSongs = data.loc[:,"Lyrics"]
print(len(allSongs))
createTextFile(corpusName = "HindiCorpus.txt", data = allSongs)
    


1291


## Create Bengali Corpora


In [20]:
#returns an array with all the strings 
def readCSV(dataFile,columnName ): 
    """
    readCSV reads a CSV and loads data into an array which is returned
    @parameters:
    :dataFile:= path to the CSV to be read 
    :columnName:= name of the column where our data of interest lies 
    :returns:= the data as an array
    """
    data = pd.read_csv(dataFile)
    df = pd.DataFrame(data)
    allSongs = data.loc[:, columnName]
    return allSongs 



In [4]:
allSongs = readCSV(dataFile="BanglaSongLyrics.csv", columnName="lyrics")

In [5]:
print(allSongs[:5])

0    এবার তোমার কথাগুলোর\nকাছে আমি অপরাজিত\nজমে থাক...
1    আমার ভায়ের রক্তে রাঙানো একুশে ফেব্রুয়ারি\nআমি ...
2    আমি কুকুর হইয়া দেই পাহাড়া\nদয়াল চান্দের ঘড় বাড়...
3    তুমি এসমে আজ যত তরানে ওলা\nগাউছেলাম বাবা নুরে ...
4    আমার আপনার চেয়ে আপন যে জন\nখুঁজি তারে আমি আপন...
Name: lyrics, dtype: object


In [11]:
createTextFile(corpusName="banglaNativeCorpus.txt", data=allSongs)

cn banglaNativeCorpus.txt


## Scrape Native Hindi from csv into txt

In [51]:
allHindiSongs = readCSV(dataFile="hindiNativeCorpus.csv", columnName="hindi_sentence")

In [54]:
createTextFile(corpusName="hindiNativeCorpus.txt", data=allHindiSongs)

cn hindiNativeCorpus.txt


## Get English Data

In [49]:
english_text = open('downloads/englishCorpus.txt').readlines()

In [50]:
cleaned_english_text = [line.split('\n')[0] for line in english_text]
cleaned_english_text = list(filter(lambda x: x, cleaned_english_text))#rids empty string 
print(len(cleaned_english_text))


10144


In [79]:
imdb_text = readCSV(dataFile="downloads/IMDB Dataset.csv", columnName="review")[:100000]
cleaned_imdb = [line.split('\n')[0] for line in imdb_text]
import string 
eng = []
for line in cleaned_imdb: 
    res = line
    res= res.split('\n')[0]
    res = re.sub(re.compile('<.*?>'), '', res)
#     res = res.join(res.split("."))
    for s in res.split("."):
        eng.append(s) 

# cleaned_imdb = [line.translate(str.maketrans('', '', string.punctuation)) for line in imdb_text]


In [77]:
def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

In [81]:
createTextFile(corpusName="englishCorpus.txt", data=eng)

cn englishCorpus.txt
